In [252]:
from SoundsLike.SoundsLike import Search
from PyDictionary import PyDictionary
import pandas as pd
dictionary=PyDictionary()
import nltk
import re
nltk.download('punkt')
nltk.download('universal_tagset')
from nltk.wsd import lesk
import eng_to_ipa as ipa
import os
import openai
import numpy as np
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pbhusari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\pbhusari\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [39]:
header_list = ['word', 'punStatus', 'pronunciation']
ingestionDataFrameHete = pd.read_csv("./concatTrainHete.txt", sep=" ", names=header_list)

gpt3dataFrameHete = pd.DataFrame(columns=["prompt", "punWord"])

prompt = ""
punWord = ""

for index, row in ingestionDataFrameHete.iterrows():
    word = row["word"]
    punStatus = row["punStatus"]

    if word == "<sentEnd>":
        prompt = prompt.replace("\'\'", "\"")
        prompt = prompt.replace("\' ", "\'")
        prompt = prompt.replace("\" ", "\"")
        prompt = prompt.replace("- ", "-")
        prompt = prompt.replace(',"', '," ')
        prompt = prompt.replace(',\'', ',\' ')
        appendDict = {"prompt":prompt, "punWord":punWord}
        tempAddDf = pd.DataFrame([appendDict])
        gpt3dataFrameHete = pd.concat([gpt3dataFrameHete, tempAddDf], ignore_index=True)

        prompt = ""
        punWord = ""
    else:
        if "PUNCTUATION" in row["pronunciation"]:
            prompt = prompt + word
        else:
            prompt = prompt + " " + word
        if punStatus == "P":
            punWord = word
gpt3dataFrameHete

,prompt,punWord
0,"""I'm halfway up a mountain,"" Tom alleged.",alleged
1,"I'd like to be a Chinese laborer, said Tom co...",coolly
2,Dentists don't like a hard day at the orifice.,orifice
3,Are evil wildebeests bad gnus?,gnus
4,A busy barber is quite harried.,harried
...,...,...
1267,I felt sick after I ate the scallopini. I did...,veal
1268,Church disputes can be settled with canons.,canons
1269,"I love the novels of D. H. Lawrence, said the...",chattily
1270,"She was only a Carpenter's daughter, but you ...",sores


In [262]:
sentence = "Joke: Old electricians never die, they just do it until it hertz.".lower()
targetlen = 6


response = openai.Completion.create(
      model="davinci:ft-personal-2022-10-02-22-42-09",
      prompt=sentence + "->",
)
jokeWord = re.sub(r'[^a-zA-Z0-9]', '', response["choices"][0]["text"][0:targetlen])
print(jokeWord)

hertz


In [263]:

jokeWord = re.sub(r'[^a-zA-Z0-9]', '', jokeWord.lower())

for word,pos in nltk.tag.pos_tag(sentence.split(" "), tagset='universal'):
    word = re.sub(r'[^a-zA-Z0-9]', '', word)
    if word == jokeWord:
        break

posCanidateList = []
for item in Search.closeHomophones(jokeWord):
    wordDef = dictionary.meaning(item)
    if wordDef is not None:
        if str(wordDef)[0:5] != "Error:":
            posCanidateList.append((item, wordDef))

universalTagDict = {
    "ADJ":"Adjective",
    "ADV":"Adverb",
    "NOUN":"Noun",
    "PRON":"Noun",
    "VERB":"Verb"
}

if pos in universalTagDict.keys():
    pos = universalTagDict[pos]


for word, definition in posCanidateList:
    if word == jokeWord.capitalize():
        pass
    else:
        for item in definition:
            if pos == str(item):
                print(word, definition[item][0])



Error: The Following Error occured: list index out of range
Hurts be the source of pain


In [264]:
openai.api_key = 'sk-'

openAiExplanation = openai.Completion.create(
  model="text-davinci-002",
  prompt="Joke: " + sentence.capitalize() + "\nSource: "+ jokeWord +"\nTarget: " + word.lower() + "\nExplanation: ",
  temperature=0.15,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)


In [265]:
sourceDefinition = lesk(sentence, jokeWord).definition()
targetDefinition = lesk(sentence.replace(jokeWord, word.lower()), word).definition()

resp = openai.Embedding.create(
    input=[sourceDefinition, targetDefinition],
    engine="text-similarity-davinci-001")

embedding_a = resp['data'][0]['embedding']
embedding_b = resp['data'][1]['embedding']

definitionSimilarityScore = np.dot(embedding_a, embedding_b)
pronounciationSimilarityScore = nltk.edit_distance(ipa.convert(jokeWord), ipa.convert(word.lower()))

jokeScore = 100 - (80 * sigmoid(pronounciationSimilarityScore) + 20 * definitionSimilarityScore)


In [266]:
print("Joke: " + sentence.capitalize())
print("1.")
print("___________________")
print("Source: " + jokeWord)
print("Target: " + word.lower())
print("2.")
print("___________________")
# Compute sound similarity or difference (you may use one of the metrics you tried in class exercise.
print("IPA Edit Distance (Method 1): " + str(pronounciationSimilarityScore))
print("3.")
print("___________________")
print("Source Definition: " + sourceDefinition)
print("Target Definition: " + targetDefinition)
print("4.")
print("___________________")
# Explain what makes it a joke/pun based on the similarity of meanings and pronunciation.
print("Explanation: " + str(openAiExplanation['choices'][0]['text'].strip()))
# GPT-3
print("5.")
print("___________________")
# Bonus: come up with a measure of how good a joke/pun is based on your appreciation of it, sound similarity of target/source; meaning similarity, or anything else that you feel is useful.
# Just composite phoneme edit distance (0.8) and lesk definition similarity (0.2)
# Sound similar, different definitions
print("Joke Score: " +  str(jokeScore))

Joke: Joke: old electricians never die, they just do it until it hertz.
1.
___________________
Source: hertz
Target: hurts
2.
___________________
IPA Edit Distance (Method 1): 1
3.
___________________
Source Definition: the unit of frequency; one hertz has a periodic interval of one second
Target Definition: a damage or loss
4.
___________________
Explanation: This joke is a play on words. The word "hertz" is a unit of measurement for frequency, and it sounds like the word "hurts." The joke is saying that old electricians never die, they just keep working until it hurts.
5.
___________________
Joke Score: 28.323935454985374
